In [1]:
import os
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import multiprocessing
from os import walk
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [2]:
RESULTS_DIR = './svm_results/'
label_map = {'Hold': 0, 'Buy': 1, 'Sell': 2}
label_map_reverse = {0:'Hold', 1:'Buy', 2:'Sell'}

In [3]:
def get_paths(filepath):
    dataset_paths = []
    labels_paths = []
    stock_names = []
    for (dirpath, dirnames, filenames) in walk(filepath):
        dataset_paths = [filepath + '/' + name for name in dirnames]
        labels_paths = [filepath + '/' + name for name in filenames if name != '.DS_Store']
        stock_names = [name.split('_')[0] for name in filenames if name != '.DS_Store']
        break
    dataset_paths.sort()
    labels_paths.sort()
    stock_names.sort()

    return dataset_paths, labels_paths, stock_names

In [4]:
def read_data(d_path, l_path, s_name):
    
    with open(d_path+'/data.pkl', 'rb') as f:
        dataset = pickle.load(f)
    
    labels = np.zeros(len(dataset))
    prices = np.zeros(len(dataset))
    dates = np.empty(len(dataset), dtype='datetime64[s]')
    with open(l_path, 'r') as f:
        for i, line in enumerate(f):
            info = line.strip().split(', ')
            dates[i] = np.datetime64(info[1])
            prices[i] = info[2]
            labels[i] = label_map[info[3]]

    return [dataset, dates, prices, labels]

In [5]:
def data_augmentation(X_data, Y_data):
    proportion = np.array([0.34, 0.33, 0.33])
    labels_ind = [np.where(Y_data == x)[0] for x in range(3)]
    labels_tot = math.ceil(max([labels_ind[x].shape[0] / proportion[x] for x in range(len(labels_ind))]))

    labels_num = (proportion * labels_tot).astype(int)
    labels = np.arange(labels_num.sum())
    
    l = Y_data.shape[0]
    for i in range(labels_num.shape[0]):
        additional_ind = np.random.choice(labels_ind[i], labels_num[i] - labels_ind[i].shape[0])
        r = l + additional_ind.shape[0]
        labels[l:r] = additional_ind
        l = r

    return X_data[labels], Y_data[labels]

In [6]:
def data_processing(dataset, labels, augmentation):
    if augmentation:
        dataset, labels = data_augmentation(dataset, labels)
    
    X_data = dataset
    Y_data = labels
    input_shape = X_data.shape[1:]

    return X_data, Y_data, input_shape

In [7]:
def data_extraction(data, start_year, end_year, augmentation):
    r = np.argwhere((data[1] >= np.datetime64(str(start_year))) & (data[1] < np.datetime64(str(end_year))))
    s = r[0][0]
    e = r[-1][0]

    dataset = data[0][r[:, 0]]
    dates = data[1][r[:, 0]]
    prices = data[2][r[:, 0]]
    labels = data[3][r[:, 0]]

    return data_processing(dataset, labels, augmentation), dates, prices

In [8]:
def train(X_train, Y_train, X_test, Y_test, batch_size, epochs):
    model = SVC(class_weight='balanced')
    model.fit(X_train, Y_train)

    return model

In [9]:
def test(model, X_test, Y_test):
    y_pred = model.predict(X_test)
    score = accuracy_score(Y_test, y_pred)
    return score

In [10]:
class Report:

    @staticmethod
    def write_confusion_and_classif(cmatrix, save_path, y_test, y_pred, debug=True):
        with open(save_path + "/confusion_m.txt", 'w') as file:
            out_str = "%5s %4s %4s %4s" % ("", label_map_reverse[0], label_map_reverse[1], label_map_reverse[2])
            file.write(out_str + '\n')
            if debug:
                    print(out_str)

            for i, row in enumerate(cmatrix):
                out_str = "%4s: %4d %4d %4d" % (label_map_reverse[i], row[0], row[1], row[2])
                file.write(out_str + '\n')
                if debug:
                    print(out_str)

        with open(save_path + '/report.txt', 'w') as f:
            f.write(classification_report(y_test, y_pred))
        
        return

In [11]:
def train_process(X_train, Y_train, X_test, Y_test, dates, prices, year, results_dict):
    
    model = train(X_train, Y_train, X_test, Y_test, 128, epochs=40)
    save_path = RESULTS_DIR + s_name + '/' + str(year)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    score = test(model, X_test, Y_test)
    results_dict['test_results'] += [(score, X_test.shape[0])]

    y_pred = model.predict(X_test)
    
    conf_matrix_temp = confusion_matrix(Y_test, y_pred)
    Report.write_confusion_and_classif(conf_matrix_temp, save_path, Y_test, y_pred)

    results_dict['conf_matrix'] += conf_matrix_temp
    results_dict['Y_test_sum'] = np.append(results_dict['Y_test_sum'], Y_test)
    results_dict['y_pred_sum'] = np.append(results_dict['y_pred_sum'], y_pred)
    prediction_dict = results_dict['prediction']
    for date, price, pred in zip(dates, prices, y_pred):
        prediction_dict[date] = (price, pred)
    results_dict['prediction'] = prediction_dict
    
    Report.write_confusion_and_classif(results_dict['conf_matrix'], RESULTS_DIR, results_dict['Y_test_sum'], results_dict['y_pred_sum'])
    
    with open(RESULTS_DIR + 'results.txt', 'a') as f:
        f.write('Year: {}. Number of samples: {}. Accuracy: {}.\n'.format(str(year), X_test.shape[0], score))


In [12]:
class Financial_Evaluation:

    @staticmethod
    def simulate(prediction, stock_name, years_num):
        COMMISION = 1
        
        init_balance = 10000
        balance = init_balance
        bought_cnt = 0

        for date, pred in prediction.items():
            price = pred[0]
            action = label_map_reverse[pred[1]]
            
            if action == 'Buy' and bought_cnt == 0:
                bought_cnt = (balance - COMMISION) / price
                balance = 0
            
            if action == 'Sell' and bought_cnt > 0:
                balance = bought_cnt * price - COMMISION
                bought_cnt = 0

        if bought_cnt > 0:
            balance = bought_cnt * price - COMMISION
            bought_cnt = 0

        total_return = balance / init_balance
        annualized_return = total_return ** (1.0/years_num)

        total_return = (total_return - 1.0) * 100
        annualized_return = (annualized_return - 1.0) * 100

        print('Stock:', stock_name, 'Total return:', total_return, 'Annualized return:', annualized_return)
        with open(RESULTS_DIR + 'results.txt', 'a') as f:
            f.write('Total return: {:.2f} %. Annualized return: {:.2f} %.\n'.format(total_return, annualized_return))

        return total_return, annualized_return


In [14]:
with open('./svm_results/results.txt', 'w') as f:
        f.write('')

results_dict = multiprocessing.Manager().dict()
results_dict['test_results'] = []
results_dict['Y_test_sum'] = np.array([])
results_dict['y_pred_sum'] = np.array([])
results_dict['conf_matrix'] = np.zeros((3, 3))
results_dict['prediction'] = {}
images_paths, labels_paths, stock_names = get_paths('./data')

ann_return = []
for s_ind, (i_path, l_path, s_name) in enumerate(zip(images_paths, labels_paths, stock_names)):
    print("STOCK: %s" % s_name)
    with open(RESULTS_DIR + 'results.txt', 'a') as f:
            f.write('\nSTOCK: {}\n'.format(s_name))

    data = read_data(i_path, l_path, s_name)

    results_dict['prediction'] = {}

    start_year = 2007
    years_num = 10
    for i in range(years_num):
        print('Testing year: ' + str(start_year + i))

        (X_train, Y_train, input_shape), _, _ = data_extraction(data, (start_year - 5) + i, start_year + i, False)
        (X_test, Y_test, input_shape), dates, prices = data_extraction(data, start_year + i, (start_year + 1) + i, False)

        p = multiprocessing.Process(target=train_process, args=(X_train, Y_train, X_test, Y_test, dates, prices, start_year + i, results_dict))
        p.start()
        p.join()


    total_return, annualized_return = Financial_Evaluation.simulate(results_dict['prediction'], s_name, years_num)
    ann_return.append(annualized_return)


print('Average annualized return:', sum(ann_return) / len(ann_return))
# printing results
num_samples = sum([x[1] for x in results_dict['test_results']])
total_loss = sum([x[0] * x[1] for x in results_dict['test_results']]) / num_samples
total_acc = sum([x[1] * x[1] for x in results_dict['test_results']]) / num_samples
with open(RESULTS_DIR + 'results.txt', 'a') as f:
    f.write('\nNumber of samples: {}. Total Loss: {}. Total accuracy: {}.\n'.format(num_samples, total_loss, total_acc))

STOCK: AAPL
Testing year: 2007
      Hold  Buy Sell
Hold:  146   70   10
 Buy:    5    7    0
Sell:   11    1    1
      Hold  Buy Sell
Hold:  146   70   10
 Buy:    5    7    0
Sell:   11    1    1
Testing year: 2008
      Hold  Buy Sell
Hold:   46  174   10
 Buy:    1   12    0
Sell:    1    7    2
      Hold  Buy Sell
Hold:  192  244   20
 Buy:    6   19    0
Sell:   12    8    3
Testing year: 2009
      Hold  Buy Sell
Hold:   55  104   67
 Buy:    2   11    1
Sell:    4    3    5
      Hold  Buy Sell
Hold:  247  348   87
 Buy:    8   30    1
Sell:   16   11    8
Testing year: 2010
      Hold  Buy Sell
Hold:    0  189   34
 Buy:    0   14    0
Sell:    0   10    5


/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


      Hold  Buy Sell
Hold:  247  537  121
 Buy:    8   44    1
Sell:   16   21   13
Testing year: 2011
      Hold  Buy Sell
Hold:    5  205   12
 Buy:    0   16    0
Sell:    0   12    2
      Hold  Buy Sell
Hold:  252  742  133
 Buy:    8   60    1
Sell:   16   33   15
Testing year: 2012
      Hold  Buy Sell
Hold:    0  160   64
 Buy:    0   12    2
Sell:    0    8    4


/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


      Hold  Buy Sell
Hold:  252  902  197
 Buy:    8   72    3
Sell:   16   41   19
Testing year: 2013
      Hold  Buy Sell
Hold:    0   63  156
 Buy:    0    8   10
Sell:    0    2   13


...

      Hold  Buy Sell
Hold: 11866 9777 33101
 Buy:  936 1141 1803
Sell:  722  360 2740
Testing year: 2015
      Hold  Buy Sell
Hold:   86    1  134
 Buy:    8    0    9
Sell:    4    0   10
      Hold  Buy Sell
Hold: 11952 9778 33235
 Buy:  944 1141 1812
Sell:  726  360 2750
Testing year: 2016
      Hold  Buy Sell
Hold:  175    3   44
 Buy:   16    0    2
Sell:    8    0    4
      Hold  Buy Sell
Hold: 12127 9781 33279
 Buy:  960 1141 1814
Sell:  734  360 2754
Stock: UTX Total return: 9.011444498046451 Annualized return: 0.8665599411535663
STOCK: VZ
Testing year: 2007
      Hold  Buy Sell
Hold:   76   21  121
 Buy:    7    5    5
Sell:    4    0   12
      Hold  Buy Sell
Hold: 12203 9802 33400
 Buy:  967 1146 1819
Sell:  738  360 2766
Testing year: 2008
      Hold  Buy Sell
Hold:   83   88   44
 Buy:    8    9    3
Sell:    4    5    9
      Hold  Buy Sell
Hold: 12286 9890 33444
 Buy:  975 1155 1822
Sell:  742  365 2775
Testing year: 2009
      Hold  Buy Sell
Hold:  110   65   50
 Buy: 

/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


      Hold  Buy Sell
Hold: 13074 10066 33605
 Buy: 1017 1176 1825
Sell:  791  370 2788
Testing year: 2014
      Hold  Buy Sell
Hold:   11   27  181
 Buy:    0    4   11
Sell:    1    1   16
      Hold  Buy Sell
Hold: 13085 10093 33786
 Buy: 1017 1180 1836
Sell:  792  371 2804
Testing year: 2015
      Hold  Buy Sell
Hold:   13   32  175
 Buy:    2    7    9
Sell:    1    0   13
      Hold  Buy Sell
Hold: 13098 10125 33961
 Buy: 1019 1187 1845
Sell:  793  371 2817
Testing year: 2016
      Hold  Buy Sell
Hold:   11   51  167
 Buy:    2    5    5
Sell:    0    0   11
      Hold  Buy Sell
Hold: 13109 10176 34128
 Buy: 1021 1192 1850
Sell:  793  371 2828
Stock: VZ Total return: 87.4772950260461 Annualized return: 6.486577237492286
STOCK: WMT
Testing year: 2007
      Hold  Buy Sell
Hold:   11   36  168
 Buy:    1    7   10
Sell:    1    1   16
      Hold  Buy Sell
Hold: 13120 10212 34296
 Buy: 1022 1199 1860
Sell:  794  372 2844
Testing year: 2008
      Hold  Buy Sell
Hold:    9   24  180
 Bu

/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


      Hold  Buy Sell
Hold: 13676 10588 37555
 Buy: 1075 1260 2046
Sell:  820  385 3099
Stock: XOM Total return: 205.22147591808744 Annualized return: 11.805072820079276
Average annualized return: 4.899798042944797
